# Homework 9

## Exercise 1

In [3]:
import heapq
import numpy as np

from collections import deque
from enum import Enum
from PIL import Image
from pqdict import minpq
from time import perf_counter as timer

In [170]:
class Color(Enum):
    WHITE = (255, 255, 255)
    PINK = (255, 192, 203)
    BLUE = (0, 0, 255)
    RED = (255, 0, 0)
    LIGHTBLUE = (173, 216, 230)
    GREEN = (144, 238, 144)

In [158]:
def find_neighbours(image, u):
    neighbours = []
    
    for i in range(-1, 2):
        for j in range(-1, 2):
            
            # not u
            if i == j == 0:
                continue

            v = (u[0] + i, u[1] + j)
            # verify that we don't cross the bounds
            if (v[0] < 0 or v[0] >= image.width) or \
               (v[1] < 0 or v[1] + j >= image.height):
                continue
                
            if i == j or i == -j:
                # euclidean distance for diagonal neighbours
                dist = np.sqrt(2)
            else:
                dist = 1
                
            if Color(image.getpixel(v)) == Color.PINK:
                # wall
                dist = np.inf
            elif Color(image.getpixel(v)) == Color.GREEN:
                # swamp
                dist *= 2
            elif Color(image.getpixel(v)) == Color.LIGHTBLUE:
                # sea
                dist *= 4
            
            neighbours.append((v, dist))
    
    return neighbours

In [163]:
def dijkstra_algorithm(image, origin, target):
    prev = np.full(image.size, None)
    dist = np.full(image.size, np.inf)
    queue = minpq()
    
    dist[origin] = 0
    queue[origin] = 0
    
    for u, min_dist in queue.popitems():
        if u == target:
            break
        
        for v, edge in find_neighbours(image, u):
            alt = dist[u] + edge

            if alt < dist[v]:
                dist[v] = alt
                queue[v] = alt
                prev[v] = u
        
    return dist, prev

In [164]:
def get_path(prev, target):
    path = deque([])
    current = target
    
    while current is not None:
        path.appendleft(current)
        current = prev[current]
    
    return path

In [109]:
def get_coverage(dist):
    visited = []
    
    for i in range(dist.shape[0]):
        for j in range(dist.shape[1]):
            if dist[i][j] != np.inf:
                visited.append((i, j))
    
    return visited

In [110]:
def find_origin_target(image):
    origin = target = None
    
    for i in range(image.height): 
        for j in range(image.width):
            if Color(image.getpixel((i, j))) == Color.BLUE:
                origin = (i, j)
                if target is not None:
                    return (origin, target)
            elif Color(image.getpixel((i, j))) == Color.RED:
                target = (i, j)
                if origin is not None:
                    return (origin, target)
    
    return (origin, target)

In [165]:
with Image.open('map250.png').convert('RGB') as im:
    origin, target = find_origin_target(im)
    
    start = timer()
    dist, prev = dijkstra_algorithm(im, origin, target)
    end = timer()
    
    print(f"Dijkstra algorithm execution time: {round(end - start, 3)} s.")
    print(f"Shortest distance from A to B by Dijkstra algorithm: {round(dist[target], 3)}.")
    
    with open("1/dijkstra_path.txt", "w") as file:
        path = get_path(prev, target)
        for v in path:
            file.write(f"{v[0]} {v[1]}\n")
    
    with open("1/dijkstra_coverage.txt", "w") as file:
        coverage = get_coverage(dist)
        for v in coverage:
            file.write(f"{v[0]} {v[1]}\n")

Dijkstra algorithm execution time: 3.03 s.
Shortest distance from A to B by Dijkstra algorithm: 637.328.


The path:
<img src='1/dijkstra_path.png'>

The coverage:
<img src='1/dijkstra_coverage.png'>

Dijkstra algorithm finds the shortest path in 3 seconds but needs to check almost all pixels. Let's try A* and see whether introducing heuristics will help to decrease coverage.

## Exercise 2

In [4]:
def euclidean_distance(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

In [166]:
def a_star_algorithm(image, origin, target):
    prev = np.full(image.size, None)
    dist = np.full(image.size, np.inf)
    queue = minpq()
    
    dist[origin] = 0
    queue[origin] = 0
    
    for u, min_dist in queue.popitems():
        if u == target:
            break
        
        for v, edge in find_neighbours(image, u):
            alt = dist[u] + edge

            if alt < dist[v]:
                dist[v] = alt
                queue[v] = alt + euclidean_distance(v, target)
                prev[v] = u
        
    return dist, prev

In [167]:
with Image.open('map250.png').convert('RGB') as im:
    origin, target = find_origin_target(im)
    
    start = timer()
    dist, prev = a_star_algorithm(im, origin, target)
    end = timer()
    
    print(f"A*-algorithm execution time: {round(end - start, 3)} s.")
    print(f"Shortest distance from A to B by A*-algorithm: {round(dist[target], 3)}.")
    
    with open("2/a_star_path.txt", "w") as file:
        path = get_path(prev, target)
        for v in path:
            file.write(f"{v[0]} {v[1]}\n")
    
    with open("2/a_star_coverage.txt", "w") as file:
        coverage = get_coverage(dist)
        for v in coverage:
            file.write(f"{v[0]} {v[1]}\n")

A*-algorithm execution time: 2.87 s.
Shortest distance from A to B by A*-algorithm: 637.328.


The path:
<img src='2/a_star_path.png'>

The coverage:
<img src='2/a_star_coverage.png'>

So, introducing heuristics (euclidean distance) in A* algorithm, despite the extra amount of computations, resulted in better performance and smaller coverage.

## Exercise 3

In [171]:
with Image.open('map1000.png').convert('RGB') as im:
    origin, target = find_origin_target(im)
    
    start = timer()
    dist, prev = a_star_algorithm(im, origin, target)
    end = timer()
    
    print(f"A*-algorithm execution time: {round(end - start, 3)} s.")
    print(f"Shortest distance from A to B by A*-algorithm: {round(dist[target], 3)}.")
    
    with open("3/a_star_path.txt", "w") as file:
        path = get_path(prev, target)
        for v in path:
            file.write(f"{v[0]} {v[1]}\n")
    
    with open("3/a_star_coverage.txt", "w") as file:
        coverage = get_coverage(dist)
        for v in coverage:
            file.write(f"{v[0]} {v[1]}\n")

A*-algorithm execution time: 55.52 s.
Shortest distance from A to B by A*-algorithm: 2002.344.


The path:
<img src='3/a_star_path.png' width='600'>

The coverage:
<img src='3/a_star_coverage.png' width='600'>

As we can see it from the map, algorithm decides to come around the swamps and traverse the sea (choosing the minimum trajectory of traversal). Avoiding the swamps apparently is more efficient than traversing them, hence the above path we get.

## Exercise 4

Let's implement the classical nearest neighbour algorithm, though run it N times, choosing all cities as an origin, and pick up the best result.

In [19]:
class Graph:
    
    def __init__(self, vertices_num):
        self.vertices = []
        self.matrix = np.zeros((vertices_num, vertices_num))
        
    def vertex(self, i):
        return self.vertices[i]
    
    def add_edge(self, v1, v2, dist):
        self.matrix[v1][v2] = dist
        self.matrix[v2][v1] = dist
    
    def parse(self, file):
        for i, line in enumerate(file.readlines()):
            v1 = [int(a) for a in line.split()]
            for j, v2 in enumerate(self.vertices):
                self.add_edge(i, j, euclidean_distance(v1, v2))
            self.vertices.append(v1)
    
    def size(self):
        return len(self.vertices)
    
    def edges_num(self):
        return sum([len(lst) for lst in self.adjacent]) // 2
    
    def dist(self, v1, v2):
        return self.matrix[v1][v2]

In [24]:
def nearest_neighbour_algorithm(graph, origin):
    path = [origin]
    visited = [False] * graph.size()
    visited[origin] = True
    i = origin

    for _ in range(graph.size() - 1):
        nearest = None
        min_dist = np.inf

        for j in range(graph.size()):
            dist = graph.dist(i, j)
            if not visited[j] and dist < min_dist:
                nearest = j
                min_dist = dist

        path.append(nearest)
        visited[nearest] = True
        i = nearest
    
    return path

In [37]:
def calc_total_distance(graph, path):
    return sum([graph.dist(path[i - 1], path[i]) for i in range(len(path))])

In [56]:
def NN_shortest_path(graph):
    paths = []
    
    for i in range(graph.size()):
        paths.append(nearest_neighbour_algorithm(graph, i))
    distances = [calc_total_distance(graph, path) for path in paths]
    min_index = np.argmin(distances)
    
    return paths[min_index], distances[min_index]

In [14]:
import requests

from zipfile import ZipFile

In [8]:
with open('tsp.zip', 'wb') as file:
    file.write(requests.get('https://courses.cs.ut.ee/2022/algorithmics/uploads/Main/tsp.zip').content)

with ZipFile('tsp.zip', 'r') as zip:
    zip.extractall()
    zip.printdir()

File Name                                             Modified             Size
10.txt                                         2020-10-29 19:15:36           88
100.txt                                        2020-10-29 19:16:48          881
20.txt                                         2020-10-29 19:16:16          178


In [57]:
sizes = [10, 20, 100]

for size in sizes:
    graph = Graph(size)
    
    with open(str(size) + '.txt', 'r') as file:
        graph.parse(file)
    
    start = timer()
    path, dist = NN_shortest_path(graph)
    end = timer()
    print(f"NN-Algorithm on {size} cities finished execution in {end - start} s.")
    print(f"Total distance: {round(dist, 3)}.")
    print(f"Shortes tour: {path}.\n")
    
    with open('4/path' + str(size) + '.txt', 'w') as file:
        for i in path:
            file.write(f"{i}\n")

NN-Algorithm on 10 cities finished execution in 0.0006376000001182547 s.
Total distance: 2261.663.
Shortes tour: [2, 0, 4, 8, 1, 3, 6, 5, 9, 7].

NN-Algorithm on 20 cities finished execution in 0.0031053000002430053 s.
Total distance: 4479.171.
Shortes tour: [4, 9, 7, 12, 10, 11, 5, 15, 17, 18, 1, 19, 16, 3, 14, 0, 13, 2, 6, 8].

NN-Algorithm on 100 cities finished execution in 0.337077500000305 s.
Total distance: 8449.27.
Shortes tour: [32, 30, 13, 34, 1, 49, 75, 64, 68, 66, 76, 56, 48, 63, 40, 39, 91, 43, 84, 82, 21, 89, 65, 94, 80, 22, 58, 46, 54, 78, 2, 26, 9, 33, 28, 98, 57, 8, 3, 69, 61, 83, 67, 71, 62, 6, 86, 7, 97, 14, 60, 5, 27, 47, 52, 23, 25, 79, 17, 45, 95, 10, 20, 77, 0, 74, 29, 24, 88, 4, 85, 18, 59, 44, 11, 42, 51, 19, 37, 36, 31, 90, 92, 50, 99, 93, 70, 53, 81, 15, 12, 73, 87, 38, 41, 55, 72, 35, 96, 16].



**10 cities shortest path:**
<img src='4/path10.png' width='600'>

**20 cities shortest path:**
<img src='4/path20.png' width='600'>

**100 cities shortest path:**
<img src='4/path100.png' width='600'>

## Exercise 5

In [40]:
np.random.seed(1111)

In [46]:
def find_path_neighbours(path):
    neighbours = []
    
    for i in range(len(path)):
        for j in range(i + 1, len(path)):
            neighbour = path.copy()
            # swap cities in routes
            neighbour[i], neighbour[j] = path[j], path[i]
            neighbours.append(neighbour)
    
    return neighbours

In [47]:
def get_best_neighbour(graph, neighbours):
    min_dist = np.inf
    min_path = None
    
    for neighbour in neighbours:
        dist = calc_total_distance(graph, neighbour)
        if dist < min_dist:
            min_dist = dist
            min_path = neighbour
    
    return min_path, min_dist

In [48]:
def hill_climbing_algorithm(graph, origin):
    current = list(range(graph.size()))
    np.random.shuffle(current)
    
    current_dist = calc_total_distance(graph, current)
    neighbours = find_path_neighbours(current)
    best_neighbour, best_dist = get_best_neighbour(graph, neighbours)

    while best_dist < current_dist:
        current = best_neighbour
        current_dist = best_dist
        neighbours = find_path_neighbours(current)
        best_neighbour, best_dist = get_best_neighbour(graph, neighbours)

    return current, current_dist

In [49]:
sizes = [10, 20, 100]

for size in sizes:
    graph = Graph(size)
    
    with open(str(size) + '.txt', 'r') as file:
        graph.parse(file)
    
    start = timer()
    path, dist = hill_climbing_algorithm(graph)
    end = timer()
    print(f"Hill Climbing Algorithm on {size} cities finished execution in {end - start} s.")
    print(f"Total distance: {round(dist, 3)}.")
    print(f"Shortes tour: {path}.\n")
    
    with open('5/path' + str(size) + '.txt', 'w') as file:
        for i in path:
            file.write(f"{i}\n")

Hill Climbing Algorithm on 10 cities finished execution in 0.0020893000000796746 s.
Total distance: 2270.988.
Shortes tour: [2, 0, 4, 1, 8, 7, 9, 5, 6, 3].

Hill Climbing Algorithm on 20 cities finished execution in 0.019010600000001432 s.
Total distance: 4912.013.
Shortes tour: [5, 15, 17, 18, 1, 3, 14, 0, 13, 12, 7, 9, 4, 2, 8, 6, 16, 19, 10, 11].

Hill Climbing Algorithm on 100 cities finished execution in 24.06197599999996 s.
Total distance: 12932.252.
Shortes tour: [12, 73, 13, 30, 1, 34, 87, 16, 96, 35, 72, 55, 38, 41, 32, 61, 83, 67, 71, 43, 84, 39, 40, 51, 19, 42, 23, 47, 27, 5, 9, 26, 2, 22, 80, 94, 65, 95, 52, 45, 17, 79, 25, 91, 8, 33, 28, 98, 57, 36, 37, 31, 90, 92, 53, 70, 59, 18, 66, 56, 62, 3, 69, 6, 86, 82, 60, 14, 21, 89, 46, 58, 54, 78, 97, 7, 48, 63, 76, 68, 64, 49, 75, 44, 11, 24, 88, 20, 10, 77, 4, 74, 0, 29, 85, 50, 99, 93, 81, 15].



**10 cities shortest path:**
<img src='5/path10.png' width='600'>

**20 cities shortest path:**
<img src='5/path20.png' width='600'>

**100 cities shortest path:**
<img src='5/path100.png' width='600'>

Compared to Nearest Neighbour algorithm, Hill Climbing algorithm worked slower and provided worse solutions. On a graph with 100 cities, the difference in results and execution times became significant. Perhaps, by generating some other random set-up, we may achieve better results for Hill Climbing. Also, we can modify the strategy of choosing the neighbourhood, this may help us achieve better performance (but there will still be a trade-off between optimality of a solution and execution time).

Let's try to run 100 iterations of Hill Climbing on 20 cities to see how randomization may influence the results.

In [62]:
min_path = None
min_dist = np.inf
times = []
n_iterations = 100

graph = Graph(20)
with open('20.txt', 'r') as file:
    graph.parse(file)

for _ in range(n_iterations):
    start = timer()
    path, dist = hill_climbing_algorithm(graph)
    end = timer()
    times.append(end - start)
    
    if dist < min_dist:
        min_dist = dist
        min_path = path

print(f"Average time running Hill Climbing algorithm on 20 cities: {np.mean(times)} s.")
print(f"Best found solution: {round(min_dist, 3)}.")
print(f"Running time of Nearest Neighbour algorithm: 0.0031053000002430053 s.")
print(f"Best solution of Nearest Neighbour algorithm: 4479.171.")

Average time running Hill Climbing algorithm on 20 cities: 0.027038689000019076 s.
Best found solution: 4113.342.
Running time Nearest Neighbour algorithm: 0.0031053000002430053 s.
Best solution of Nearest Neighbour algorithm: 4479.171.


So, despite much bigger running time, Hill Climbing may return even better result than Nearest Neighbour algorithm.